In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re
from time import sleep

In [ ]:
#get all html
MARKETPLACES = []
with open("marketplaces_hina.csv") as fin: #list of 1000+ marketplaces filtered by Hina
    for line in fin:
        MARKETPLACES.append(line.split(',')[0])


raw_data = []
for mp in MARKETPLACES:
    print("Retrieving html for", mp)
    try:
        response = requests.get("http://" + mp, timeout=10)
    except:
        print("unable to get", mp)
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    pps = soup.find_all(href=re.compile('privacy'))
    if pps!=[]:
        for pp in pps:
            raw_data.append({'website':mp, 'pp':pp['href']})
    else:
        raw_data.append({'website':mp, 'pp':'NOT_FOUND'})

df_pp = pd.DataFrame(raw_data)
df_pp.to_csv("./marketplace_pp.csv")

In [ ]:
import requests
from bs4 import BeautifulSoup


headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0"}

df = pd.read_csv('marketplace_pp_clean.csv') # clean means the urls to policies are functional
df = df[df['pp_link']!="NOT FOUND"]

for index, row in df.iterrows():
    try:
        print("fetching:" , row[1])

        file_name = row[0]+"_privacypolicy"
        url = row[1]

        path = '/home/hina/iogames/iogames_codebase/marketplace_characterization/mp_privacy_policies/'
    #     requests.adapters.DEFAULT_RETRIES = 1
        soup = BeautifulSoup(requests.get(url, timeout=10).content, "html.parser")
        text = soup.get_text()

        f = open(path+file_name, "w")
        f.write(text)
        f.close() 
    except Exception as e:
        print(e)
        pass
    
    time.sleep(60)

In [ ]:
# find all the websites where no policy was given, do an nslookup for all the websites
# and find the country of origin, see if you can find jurisdiction about the providing the privacy policy

import whois
import pycountry
import pandas as pd


dff = pd.read_csv("marketplace_pp.csv")
mp_no_pp = dff[dff['pp']=='NOT_FOUND']['website'].to_list()
L =[]
for mp in mp_no_pp:
    try:
#         print(mp)
        ccode = whois.whois(mp)['country']
        country = pycountry.countries.get(alpha_2=ccode)
        country_name = country.name
        d = {'mp':mp, "country":country_name}
        L.append(d)
    except:
        d = {'mp':mp, "country":'NOT_FOUND'}
        L.append(d)

In [ ]:
d_f = pd.DataFrame(L).sort_values('country')
d_f.to_csv("mp_no-privacypolicy_countries.csv",index = False)

In [7]:
# 1248-406 = 842 , adjust 842 - 794 = 48 websites here
import pandas as pd
d_f = pd.read_csv("mp_no-privacypolicy_countries.csv")
set(d_f['country'].to_list())
# d_f[d_f['country']=='NOT_FOUND']

{'Afghanistan',
 'Austria',
 'Bahamas',
 'Belarus',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Cayman Islands',
 'China',
 'Croatia',
 'Czechia',
 'Denmark',
 'Egypt',
 'France',
 'Germany',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Italy',
 'Japan',
 'Korea, Republic of',
 'Latvia',
 'Malta',
 'NOT_FOUND',
 'Netherlands',
 'North Macedonia',
 'Norway',
 'Panama',
 'Poland',
 'Romania',
 'Russian Federation',
 'Slovenia',
 'Spain',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Viet Nam'}

KeyError: 'country'

In [ ]:
# L= []
# for index, row in df_read_pp.iterrows():
# #     print(row[1])
#     if row[2].startswith("http"):
#         url = row[2]
#         d={"mp":row[1],'pp_link':url}
#         L.append(d)
#     elif row[2].startswith("//www"):
#         url = 'http:'+row[2]
#         d={"mp":row[1],'pp_link':url}
#         L.append(d)
#     elif row[2].startswith('/'):
#         url = "http://"+row[1]+row[2]
#         d={"mp":row[1],'pp_link':url}
#         L.append(d)
#     elif  row[2].startswith('privacy'):
#         url = "http://"+row[1]+'/'+row[2]
#         d={"mp":row[1],'pp_link':url}
#         L.append(d)
#     else:
#         pass
# L2 = [
# {'mp':  'www.cardgamesolitaire.com' , 'pp_link': 'https://www.cardgamesolitaire.com/privacy-policy.html'},
# {'mp':  'www.freegames.ws' , 'pp_link': 'https://www.freegames.ws/freegames_legal/privacy_policy.htm'},
# {'mp' : 'freewebsitetemplates.com', 'pp_link': 'https://freewebsitetemplates.com/about/privacy'},
# {'mp' : 'www.spiellen.de', 'pp_link': 'https://www.spiellen.de/privacy-policy.php'},
# {'mp' : 'www.creagames.com', 'pp_link': 'https://www.creagames.com/privacy'},
# {'mp' : 'www.freetypinggame.net', 'pp_link': 'https://www.freetypinggame.net/privacy.asp'},
# {'mp' : 'www.gleefulgames.com', 'pp_link': 'http://www.gleefulgames.com/privacy.html'},
# {'mp' : 'www.miniclip.com', 'pp_link': 'https://www.miniclip.com/games/page/en/privacy-policy#privacy-settings'},
# {'mp' : 'supermariobros.io', 'pp_link': 'NOT FOUND'},
# {'mp' : 'www.flyordie.com', 'pp_link': 'https://www.flyordie.com/legal.html#privacy'},
# {'mp' : 'www.games1.in', 'pp_link': 'https://www.games1.in/privacy-policy.php'},
# {'mp' : 'www.gamegratis33.com', 'pp_link': 'https://www.gamegratis33.com/index.php/privacy'},
# {'mp' : 'www.gamku.com', 'pp_link': 'NOT FOUND'},
# {'mp' : 'www.jouezgratuitement.fr', 'pp_link': 'https://www.jouezgratuitement.fr/privacy-policy.php'}   
# ]

# L3 = L+L2
# pd.DataFrame(L3).to_csv('marketplace_pp_clean.csv',index = False)


In [ ]:
# !pip install --trusted-host files.pythonhosted.org --trusted-host pypi.org --trusted-host pypi.python.org bs4

In [ ]:
# import requests
# from bs4 import BeautifulSoup


# headers = {
#     "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0"}

# df = pd.read_csv('marketplace_pp_clean.csv')
# df = df[df['pp_link']!="NOT FOUND"]

# for index, row in df.iterrows():
#     try:
#         print("fetching:" , row[1])

#         file_name = row[0]+"_privacypolicy"
#         url = row[1]

#         path = '/home/hina/iogames/iogames_codebase/marketplace_characterization/mp_privacy_policies_2/'
#     #     requests.adapters.DEFAULT_RETRIES = 1
#         soup = BeautifulSoup(requests.get(url, timeout=10).content, "html.parser")
#         text = soup.get_text()

#         f = open(path+file_name, "w")
#         f.write(text)
#         f.close() 
#     except Exception as e:
#         print(e)
#         pass
    
#     time.sleep(60)